In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ERM']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('ezrin radixin moesin', 405),
 ('ezrin radixin and moesin', 82),
 ('epiretinal membrane', 52),
 ('epithelial rests of malassez', 14),
 ('extraradical mycelium', 10),
 ('ezrin moesin radixin', 5),
 ('epithelial cell rests of malassez', 5),
 ('et related molecules', 4),
 ('erythromycin', 4),
 ('ezrin radixin moesin proteins', 4),
 ('egg released material', 3),
 ('er membrane', 3),
 ('enteric redmouth disease', 3)]

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'egg released material': 'ungrounded',
  'enteric redmouth disease': 'ungrounded',
  'epiretinal membrane': 'MESH:D019773',
  'epithelial cell rests of malassez': 'ungrounded',
  'epithelial rests of malassez': 'ungrounded',
  'er membrane': 'GO:GO:0005789',
  'erythromycin': 'CHEBI:CHEBI:48923',
  'et related molecules': 'ungrounded',
  'extraradical mycelium': 'MESH:D025282',
  'ezrin moesin radixin': 'FPLX:ERM',
  'ezrin radixin and moesin': 'FPLX:ERM',
  'ezrin radixin moesin': 'FPLX:ERM',
  'ezrin radixin moesin proteins': 'FPLX:ERM'},
 {'MESH:D019773': 'Epiretinal Membrane',
  'GO:GO:0005789': 'endoplasmic reticulum membrane',
  'CHEBI:CHEBI:48923': 'erythromycin',
  'MESH:D025282': 'Mycelium',
  'FPLX:ERM': 'ERM'},
 ['FPLX:ERM']]

In [10]:
grounding_map, names, pos_labels = [{'egg released material': 'ungrounded',
  'enteric redmouth disease': 'ungrounded',
  'epiretinal membrane': 'MESH:D019773',
  'epithelial cell rests of malassez': 'ungrounded',
  'epithelial rests of malassez': 'ungrounded',
  'er membrane': 'GO:GO:0005789',
  'erythromycin': 'CHEBI:CHEBI:48923',
  'et related molecules': 'ungrounded',
  'extraradical mycelium': 'MESH:D025282',
  'ezrin moesin radixin': 'FPLX:ERM',
  'ezrin radixin and moesin': 'FPLX:ERM',
  'ezrin radixin moesin': 'FPLX:ERM',
  'ezrin radixin moesin proteins': 'FPLX:ERM'},
 {'MESH:D019773': 'Epiretinal Membrane',
  'GO:GO:0005789': 'endoplasmic reticulum membrane',
  'CHEBI:CHEBI:48923': 'erythromycin',
  'MESH:D025282': 'Mycelium',
  'FPLX:ERM': 'ERM'},
 ['FPLX:ERM']]

In [11]:
excluded_longforms = []

In [12]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [13]:
additional_entities = {}

In [14]:
unambiguous_agent_texts = {}

In [15]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [1]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

NameError: name 'unambiguous_agent_texts' is not defined

In [18]:
12

In [16]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-31 01:44:32] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-31 01:45:45] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.97725830748401 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [17]:
classifier.stats

{'label_distribution': {'FPLX:ERM': 407,
  'ungrounded': 19,
  'MESH:D019773': 36,
  'CHEBI:CHEBI:48923': 4,
  'MESH:D025282': 6,
  'GO:GO:0005789': 2},
 'f1': {'mean': 0.977258, 'std': 0.009482},
 'precision': {'mean': 0.955697, 'std': 0.018278},
 'recall': {'mean': 1.0, 'std': 0.0},
 'ungrounded': {'f1': {'mean': 0.646667, 'std': 0.203961},
  'pr': {'mean': 0.516667, 'std': 0.260342},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'GO:GO:0005789': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:D025282': {'f1': {'mean': 0.733333, 'std': 0.38873},
  'pr': {'mean': 0.7, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'CHEBI:CHEBI:48923': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D019773': {'f1': {'mean': 0.956044, 'std': 0.057732},
  'pr': {'mean': 0.921429, 'std': 0.10202},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'FPLX:ERM': {'f1': {'mean': 0.977258, 'std': 0.0094

In [18]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [19]:
disamb.dump(model_name, results_path)

In [20]:
print(disamb.info())

Disambiguation model for ERM

Produces the disambiguations:
	ERM*	FPLX:ERM
	Epiretinal Membrane	MESH:D019773
	Mycelium	MESH:D025282
	endoplasmic reticulum membrane	GO:GO:0005789
	erythromycin	CHEBI:CHEBI:48923

Class level metrics:
--------------------
Grounding                     	Count	F1     
                           ERM*	407	0.97726
           Epiretinal Membrane	 36	0.95604
                    Ungrounded	 19	0.64667
                      Mycelium	  6	0.73333
                  erythromycin	  4	    0.0
endoplasmic reticulum membrane	  2	    0.2

Weighted Metrics:
-----------------
	F1 score:	0.97726
	Precision:	0.9557
	Recall:		1.0

* Positive labels
See Docstring for explanation



In [21]:
model_to_s3(disamb)